In [2]:
path = '/ML/data/sents/'
import glob
lines = []
for file in glob.glob(path+'*'):
    lines.append(' '.join(open(file).readlines()))
with open(path + 'all.txt', 'w+') as outfile:
    outfile.write(' '.join(lines))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xad in position 10: invalid start byte

In [3]:
import pydsm
cooc = pydsm.build(pydsm.CooccurrenceDSM, corpus=path + 'all.txt', window_size=(5,5), language='ru')
cooc

Building matrix from corpus with config: {'window_size': (5, 5), 'language': 'ru'}. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

CooccurrenceDSM
Vocab size: 450133
[438531, 438531]  кажо  ростовчик  исощение  ...
денис             0.00  0.00       0.00      ...
рязанцев          0.00  0.00       0.00      ...
обыкновенный      0.00  0.00       0.00      ...
...               ...   ...        ...       ...

## Задание 1 
Найдите ближайших соседей к слову "ленин":
1. по матрице совстречаемости $M$ с сырыми частотами 
2. по матрице совстречаемости $M_1$ после PPMI преобразования 
Используйте косинусную меру близости. 


In [74]:
import pydsm.similarity as similarity
cooc.nearest_neighbors('ленин', sim_func=similarity.cos)

Total time of nearest_neighbors: 5.42 sec


[438531, 1]  ленин
ленин        1.00
товарищ      0.67
сталин       0.63
памятник     0.62
гитлер       0.60
троцки       0.58
россия       0.57
народ        0.57
таки         0.57
гриб         0.56
...          ...

In [71]:
import pydsm.weighting as weighting
ppmi = cooc.apply_weighting(weight_func=weighting.ppmi)
ppmi

Total time of apply_weighting: 14.98 sec


CooccurrenceDSM
Vocab size: 450133
[438531, 438531]  наивноеа  открылок  упрыгивать  ...
денис             0.00      0.00      0.00        ...
рязанцев          0.00      0.00      0.00        ...
обыкновенный      0.00      0.00      0.00        ...
...               ...       ...       ...         ...

In [75]:
ppmi.nearest_neighbors('ленин', sim_func=similarity.cos)

Total time of nearest_neighbors: 5.65 sec


[438531, 1]  ленин
ленин        1.00
сталин       0.11
отобгать     0.11
кагтавость   0.10
бугжуев      0.10
фабгика      0.10
ильич        0.10
большевик    0.09
площадь      0.09
памятник     0.09
...          ...

## Задание 2 
Примените к матрице $M_1$ сингулярное преобразование: $M_1 = U \Sigma V^T$. Снизьте размерность до $k$.
Найдите ближайших соседей к слову "ленин":
1. по матрице $U_k $
1. по матрице $U_k \times \Sigma$
2. по матрице $U_k \times \sqrt{\Sigma}$

In [76]:
from pydsm import IndexMatrix
mat = ppmi.matrix
mat_svd = ppmi.matrix.svd(300)

In [77]:
mat_svd[0].shape, mat_svd[1].shape, mat_svd[2].shape

((438531, 300), (1, 300), (300, 438531))

In [78]:
svd_ppmi = pydsm.CooccurrenceDSM()
svd_ppmi.matrix = mat_svd[0]
svd_ppmi.vocabulary = ppmi.vocabulary
svd_ppmi

CooccurrenceDSM
Vocab size: 450133
[438531, 300]  299   298    297    ...
денис          0.01  -0.00  -0.01  ...
рязанцев       0.00  -0.00  -0.00  ...
обыкновенный   0.01  0.00   -0.00  ...
...            ...   ...    ...    ...

In [79]:
svd_ppmi.nearest_neighbors('ленин', sim_func=similarity.cos)

Total time of nearest_neighbors: 19.02 sec


[438531, 1]     ленин
ленин           1.00
подвоск         0.87
сведлов         0.84
произведний     0.84
экзорцирование  0.77
элияович        0.76
порозо          0.75
сталин          0.74
кгом            0.74
цытатко         0.73
...             ...

In [80]:
svd_ppmi = pydsm.CooccurrenceDSM()
svd_ppmi.matrix = mat_svd[0].multiply(mat_svd[1])
svd_ppmi.vocabulary = ppmi.vocabulary
svd_ppmi

CooccurrenceDSM
Vocab size: 450133
[438531, 300]  299    298    297    ...
денис          15.50  -2.72  -5.78  ...
рязанцев       3.39   -2.59  -1.62  ...
обыкновенный   27.76  5.33   -2.47  ...
...            ...    ...    ...    ...

In [81]:
svd_ppmi.nearest_neighbors('ленин', sim_func=similarity.cos)

Total time of nearest_neighbors: 11.19 sec


[438531, 1]  ленин
ленин        1.00
ильич        0.83
сталин       0.83
брежнев      0.82
памятник     0.81
хрущев       0.80
большевик    0.80
троцки       0.79
троцкий      0.77
коммунизм    0.76
...          ...

In [82]:
svd_ppmi = pydsm.CooccurrenceDSM()
svd_ppmi.matrix = mat_svd[0].multiply(mat_svd[1].sqrt())
svd_ppmi.vocabulary = ppmi.vocabulary
svd_ppmi

CooccurrenceDSM
Vocab size: 450133
[438531, 300]  299   298    297    ...
денис          0.29  -0.08  -0.20  ...
рязанцев       0.06  -0.07  -0.05  ...
обыкновенный   0.52  0.15   -0.08  ...
...            ...   ...    ...    ...

In [85]:
svd_ppmi.nearest_neighbors('ленин', sim_func=similarity.cos)

Total time of nearest_neighbors: 19.75 sec


[438531, 1]     ленин
ленин           1.00
подвоск         0.85
сведлов         0.83
произведний     0.81
экзорцирование  0.76
сталин          0.76
элияович        0.76
порозо          0.74
ильич           0.74
брежнев         0.73
...             ...

## Задание 3
Используйте word2vec и найдите ближайших соседей к слову "ленин". Сравните их с полученными ближайшими соседями на предыдущем шаге. 

In [83]:
from gensim.models import *
model = Word2Vec([line.split() for line in lines], size=300, window=5, min_count=5, workers=4)

In [84]:
model.wv.most_similar('ленин')

[('брежнев', 0.6909336447715759),
 ('троцкий', 0.6796152591705322),
 ('берия', 0.6696934700012207),
 ('тов', 0.6655898690223694),
 ('памятник', 0.6596852540969849),
 ('троцки', 0.6567761301994324),
 ('мавзолей', 0.651995062828064),
 ('энгельс', 0.6450021266937256),
 ('ильич', 0.6365503668785095),
 ('иосиф', 0.6331793665885925)]

## Задание 4
Рассмотрите другие размеры окна: 2 или 10. Как изменяются ближайшие соседи?